In [34]:
import sys
sys.path.append("../ner/BERT-NER/")

In [35]:
from bert import Ner

model = Ner("../ner/BERT-NER/out/")

In [36]:
output = model.predict("When did the House of Burgundy revolt against Philip II?")

In [4]:
output

{'When': {'tag': 'O', 'confidence': 0.9998822212219238},
 'did': {'tag': 'O', 'confidence': 0.999711811542511},
 'the': {'tag': 'O', 'confidence': 0.9998853206634521},
 'House': {'tag': 'B-ORG', 'confidence': 0.9997528195381165},
 'of': {'tag': 'I-ORG', 'confidence': 0.9949380159378052},
 'Burgundy': {'tag': 'I-ORG', 'confidence': 0.9900270104408264},
 'revolt': {'tag': 'O', 'confidence': 0.9858018755912781},
 'against': {'tag': 'O', 'confidence': 0.9998939037322998},
 'Philip': {'tag': 'B-PER', 'confidence': 0.9998753070831299},
 'II': {'tag': 'O', 'confidence': 0.9446985721588135},
 '?': {'tag': 'O', 'confidence': 0.36162233352661133}}

In [ ]:
def get_ents(sent):
    output =  model.predict(sent)
    ents = []
    ent = None
    for k, v in output.items():
        assert k is not None, k
        if v['tag'] == 'O' and ent is not None:
            ents.append(ent.lower())
            ent = None
        if v['tag'].startswith('B'):
            if ent is not None:
                ents.append(ent.lower())
            ent = k
        if v['tag'].startswith('I'):
            if ent is None:
                ent = k
            elif k.startswith("'"):
                ent += k
            else:
                ent += ' ' + k
    return ents

In [ ]:
get_ents("When did the House of Burgundy revolt against Philip II?")

In [10]:
import json
import pickle
import random

In [11]:
questions = []
contexts = []
examples = []
labels = []
id2idx = {}
files = ["dataset/train-v2.0.json",  "dataset/dev-v2.0.json"]
for file in files:
    with open(file, 'r') as handle:
        jdata = json.load(handle)
        data = jdata['data']
    for i in range(len(data)):
        section = data[i]['paragraphs']
        for sec in section:
            context = sec['context']
            contexts.append(context)
            qas = sec['qas']
            for j in range(len(qas)):
                question = qas[j]['question']
                is_imp = qas[j]['is_impossible']
                qid = qas[j]['id']
                questions.append(question)
                labels.append(is_imp)
                examples.append((len(contexts)-1, len(questions)-1))
                id2idx[qid] = len(questions)-1

In [39]:
sampled_examples_ans = []
sampled_examples_unans = []
for e in random.sample(examples, len(examples)):
    if labels[e[1]] == True and len(sampled_examples_unans) < 50:
        sampled_examples_unans.append(e)
    elif labels[e[1]] == False and len(sampled_examples_ans) < 50:
        sampled_examples_ans.append(e)
       

In [31]:
def get_ents(sent):
    output =  model.predict(sent)
    ents = {}
    ent = None
    for k, v in output.items():
        assert k is not None, k
        if v['tag'] != 'O':
            ents[k.lower()] = v['tag']
    return ents

In [42]:
ent_not_mentioned_ans = 0
ent_not_mentioned_unans = 0

# for ci, qi in sampled_examples_ans:
#     q = questions[qi]
#     c = contexts[ci]
#     cents = get_ents(c)
#     ents = get_ents(q)
#     for ent in ents.keys():
#         if ent not in cents.keys():
#             ent_not_mentioned_ans += 1
#             print("ANS", ents)
#             print(cents)
#             print(q)
#             print(c)
#             print("#"*20)
    

for ci, qi in sampled_examples_unans:
    q = questions[qi]
    c = contexts[ci]
    ents = get_ents(q)
    cents = get_ents(c)
    for ent in ents.keys():
        if ent not in cents.keys():
            ent_not_mentioned_unans += 1
            print("UNANS", ents)
            print(cents)
            print(q)
            print(c)
            print("#"*20)

print(ent_not_mentioned_ans, ent_not_mentioned_unans)

UNANS {'america': 'B-LOC', 'canadian': 'B-MISC', 'americam': 'B-MISC'}
{'canada': 'B-LOC', 'canadian': 'B-MISC', 'american': 'B-MISC'}
In America what term is used for both Canadian foobal and Americam football
In Canada, the term "football" may refer to Canadian football and American football collectively, or to either sport specifically, depending on context. The two sports have shared origins and are closely related but have significant differences. In particular, Canadian football has 12 players on the field per team rather than 11; the field is roughly 10 yards wider, and 10 yards longer between end-zones that are themselves 10 yards deeper; and a team has only three downs to gain 10 yards, which results in less offensive rushing than in the American game. In the Canadian game all players on the defending team, when a down begins, must be at least 1 yard from the line of scrimmage. (The American game has a similar "neutral zone" but it is only the length of the football.)
########

UNANS {'arsenal': 'B-ORG'}
{'the': 'B-ORG', 'art': 'B-MISC', 'deco-style': 'I-MISC', 'herbert': 'B-PER', 'chapman': 'I-PER', 'cup': 'B-MISC', 'final': 'I-MISC', 'highbury': 'B-LOC', 'stadium': 'I-LOC', 'gunners': 'I-ORG'}
What was worn on Arsenal shirts prior to 1967?
The monogram theme was developed into an Art Deco-style badge on which the letters A and C framed a football rather than the letter F, the whole set within a hexagonal border. This early example of a corporate logo, introduced as part of Herbert Chapman's rebranding of the club in the 1930s, was used not only on Cup Final shirts but as a design feature throughout Highbury Stadium, including above the main entrance and inlaid in the floors. From 1967, a white cannon was regularly worn on the shirts, until replaced by the club crest, sometimes with the addition of the nickname "The Gunners", in the 1990s.
####################
0 11


# Mask sentences

In [7]:
import pickle
import random

In [3]:
raw_targets = {}
for i in range(24):
    with open("qparts/copy_parts2/parsed_qs_labels%s.pkl"%str(i), "rb") as handle:
        raw_targets.update(pickle.load(handle))

In [4]:
from pytorch_pretrained_bert.tokenization import BertTokenizer
import spacy

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [26]:
def x_in_y_int(query, base):
    try:
        l = len(query)
    except TypeError:
        l = 1
        query = type(base)((query,))

    for i in range(len(base)):
        if base[i:i + l] == query:
            return i
    return -1


def mask_question(tokens_b, target_span):
    if not len(target_span):
        return tokens_b
    if len(target_span) > 1:
        if target_span[0] in ['ca', 'can', 'is', 'was', 'do', 'does', 'were', 'are', 'have', 'did',
                             'would', 'should', 'could'] and target_span[1] == 'n':
            new_tok = target_span[0]+target_span[1]
            del target_span[0]
            del target_span[0]
            target_span.insert(0, new_tok)
        elif target_span[0] == 'can' and target_span[1] == 'not' and 'cannot' in tokens_b:
            del target_span[0]
            del target_span[0]
            target_span.insert(0, 'cannot')
        elif target_span[0] == 'n' and target_span[1] == "'":
            del target_span[0]
#         elif target_span[0].startswith("'") and len(target_span[0]) == 2:
#             print("NOT SURE IF HTIS IS WRITE")
#             del target_span[0]
    span_start = x_in_y_int(target_span, tokens_b)
    if span_start >= 0:
        for i in range(span_start, span_start+len(target_span)):
            del tokens_b[span_start]
        tokens_b.insert(span_start, '[MASK]')
    else:
        # find first tok
        while len(target_span) > 0 and target_span[0] not in tokens_b:
            target_span = target_span[1:]
        if len(target_span) == 0:
            print("This sux")
            return tokens_b
        first_tok = tokens_b.index(target_span[0])
        for tok in tokens_b:
            if tok in target_span:
                tokens_b.remove(tok)
        tokens_b.insert(first_tok, '[MASK]')
    return tokens_b

In [28]:
for i in random.sample(raw_targets.keys(), 100000 ):
    targs = [word for word, _ in raw_targets[i]]
    idx = id2idx[i]
    question = questions[idx]
    if not targs:
        continue
    targ = random.sample(targs, 1)[0]
    x = mask_question(tokenizer.tokenize(question), targ)

This sux
This sux
This sux
This sux
This sux
This sux
This sux
This sux


In [20]:

print(question)
print(targ)
print(tokenizer.tokenize(question))
print(mask_question(tokenizer.tokenize(question), targ))

The Torah im Derech Eretz insisted that Jews should attempt to engage and influence what?
['m']
['the', 'torah', 'im', 'der', '##ech', 'er', '##etz', 'insisted', 'that', 'jews', 'should', 'attempt', 'to', 'engage', 'and', 'influence', 'what', '?']


IndexError: list index out of range

In [169]:
binascii.hexlify(bytearray(targ[0],'utf-8'))

b'2774'

In [154]:
targ[0] == "'t"

True

In [160]:
import binascii

In [168]:
binascii.hexlify(bytearray("'t",'utf-8'))

b'2774'

In [212]:
for targ in raw_targets.keys():
    if targ[0]=="'t":
        print(targ)